The <a href="https://datos.bne.es">Biblioteca Nacional de España</a> published its catalogue as Linked Open data.

This example shows how to use <a href="https://www.w3.org/TR/sparql11-query/">SPARQL</a> as a query language in Linked Open Data repositories.

### First of all, we init the SPARQLWrapper service with the SPARQL endpoint

In [21]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

### Let's write our first SPARQL sentence

We could be interested in the classes used in the repository

In [22]:
sparql.setQuery("""
PREFIX bne-def: <https://datos.bne.es/def/>
PREFIX dcterms: <http://purl.org/dc/terms/>

SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode
WHERE {
 wd:Q2085725 wdt:P950 ?id .
 wd:Q2085725 rdfs:label ?author.  FILTER(LANG(?author) = "en").            
 BIND(uri(concat("https://datos.bne.es/resource/", ?id)) as ?bneID)
 SERVICE <http://datos.bne.es/sparql> {
  ?bneID bne-def:OP5001 ?work .
  ?work rdfs:label ?workLabel .   
  OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3003 ?placeOfProduction}
  OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3006 ?yearOfPublication}
  OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition dcterms:language ?langCode}
 }
}
LIMIT 1000
"""
)

### Now we retrieve the results

In [23]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:

        if "placeOfProduction" in r:
            print(r['placeOfProduction']['value'])
            
        if "yearOfPublication" in r:
            print(r['yearOfPublication']['value'])

        if "edition" in r:
            print(r['edition']['value'])
        
        #print('Author: ', r['author']['value'], ' Work: ', r['work']['value'], 
        #      ' Title: ', r['workLabel']['value'], ' placeOfProduction: ', r['placeOfProduction']['value'],
        #      ' Year publication: ', r['yearOfPublication']['value'])
except Exception as e:
    print("Exception:")
    print(e)

London
1807
https://datos.bne.es/resource/bima0000014850
London
1760
https://datos.bne.es/resource/bima0000015581
London
1806]
https://datos.bne.es/resource/bima0000016101
Madrid
D.L. 1982
https://datos.bne.es/resource/bimo0000147143
[Boadilla del Monte]
D.L. 2012
https://datos.bne.es/resource/a5299371
Valladolid
2013
https://datos.bne.es/resource/a5535968
[Valencia]
D.L. 1996
https://datos.bne.es/resource/bimo0000695964
Te Goes
1771-1772
https://datos.bne.es/resource/a4554798
Te Goes
1771-1772
https://datos.bne.es/resource/a4554798
Paris
1776
https://datos.bne.es/resource/bima0000058106
A Nantes
1783
https://datos.bne.es/resource/bima0000058107
Madrid
D.L. 1991
https://datos.bne.es/resource/bimo0000309493
En Cadiz
1757
https://datos.bne.es/resource/bima0000058092
En Madrid
1774
https://datos.bne.es/resource/bima0000058094
Madrid
1968
https://datos.bne.es/resource/bimo0001052450
[Madrid
1941]
https://datos.bne.es/resource/bimo0001052451
Madrid
1918
https://datos.bne.es/resource/bimo000

### Let's create a dataframe

In [24]:
data = []

for r in ret["results"]["bindings"]:

    placeOfProduction = ""
    if "placeOfProduction" in r:
        placeOfProduction = r['placeOfProduction']['value']

    yearOfPublication = ""
    if "yearOfPublication" in r:
        yearOfPublication = r['yearOfPublication']['value']

    edition = ''
    if "edition" in r:
        edition = r['edition']['value']

    langCode = ''
    if "langCode" in r:
        langCode = r['langCode']['value']

    data.append({ "author" : r["author"]["value"], "work" : r["work"]["value"], 
                  "title" : r["workLabel"]["value"], "langCode" : langCode,
                  "edition" : edition, 
                  "yearOfPublication" : yearOfPublication, 
                  "placeOfProduction" : placeOfProduction})
print(data)

[{'author': 'Jorge Juan y Santacilia', 'work': 'https://datos.bne.es/resource/XX3280526', 'title': 'Relación histórica del viaje a la América Meridional', 'langCode': 'http://id.loc.gov/vocabulary/languages/eng', 'edition': 'https://datos.bne.es/resource/bima0000014850', 'yearOfPublication': '1807', 'placeOfProduction': 'London'}, {'author': 'Jorge Juan y Santacilia', 'work': 'https://datos.bne.es/resource/XX3280526', 'title': 'Relación histórica del viaje a la América Meridional', 'langCode': 'http://id.loc.gov/vocabulary/languages/eng', 'edition': 'https://datos.bne.es/resource/bima0000015581', 'yearOfPublication': '1760', 'placeOfProduction': 'London'}, {'author': 'Jorge Juan y Santacilia', 'work': 'https://datos.bne.es/resource/XX3280526', 'title': 'Relación histórica del viaje a la América Meridional', 'langCode': 'http://id.loc.gov/vocabulary/languages/eng', 'edition': 'https://datos.bne.es/resource/bima0000016101', 'yearOfPublication': '1806]', 'placeOfProduction': 'London'}, {'

In [25]:
# check placeofproduction empty!!!
#data = [{"author" : x["author"]["value"], "work" : x["work"]["value"],
#        "title" : x["workLabel"]["value"], "placeOfProduction" : x["placeOfProduction"]["value"],
#        "yearOfPublication" : x["yearOfPublication"]["value"], 
#        "langCode" : x["langCode"]["value"]} for x in ret["results"]["bindings"]]

In [26]:
import pandas as pd

df = pd.DataFrame(data)

In [27]:
df.head(10)

,author,work,title,langCode,edition,yearOfPublication,placeOfProduction
0,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280526,Relación histórica del viaje a la América Meri...,http://id.loc.gov/vocabulary/languages/eng,https://datos.bne.es/resource/bima0000014850,1807,London
1,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280526,Relación histórica del viaje a la América Meri...,http://id.loc.gov/vocabulary/languages/eng,https://datos.bne.es/resource/bima0000015581,1760,London
2,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280526,Relación histórica del viaje a la América Meri...,http://id.loc.gov/vocabulary/languages/eng,https://datos.bne.es/resource/bima0000016101,1806],London
3,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX2035522,Noticias secretas de América,http://id.loc.gov/vocabulary/languages/spa,https://datos.bne.es/resource/bimo0000147143,D.L. 1982,Madrid
4,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280515,Compendio de navegacion para el uso de los cab...,http://id.loc.gov/vocabulary/languages/spa,https://datos.bne.es/resource/a5299371,D.L. 2012,[Boadilla del Monte]
5,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280526,Relación histórica del viaje a la América Meri...,http://id.loc.gov/vocabulary/languages/spa,https://datos.bne.es/resource/a5535968,2013,Valladolid
6,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280515,Compendio de navegacion para el uso de los cab...,http://id.loc.gov/vocabulary/languages/spa,https://datos.bne.es/resource/bimo0000695964,D.L. 1996,[Valencia]
7,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280526,Relación histórica del viaje a la América Meri...,http://id.loc.gov/vocabulary/languages/dut,https://datos.bne.es/resource/a4554798,1771-1772,Te Goes
8,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280526,Relación histórica del viaje a la América Meri...,http://id.loc.gov/vocabulary/languages/dut,https://datos.bne.es/resource/a4554798,1771-1772,Te Goes
9,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280527,Dissertation historique et géographique sur le...,http://id.loc.gov/vocabulary/languages/fre,https://datos.bne.es/resource/bima0000058106,1776,Paris


In [28]:
df.describe()

,author,work,title,langCode,edition,yearOfPublication,placeOfProduction
count,39,39,39,39,39,39,39
unique,1,21,21,5,25,26,17
top,Jorge Juan y Santacilia,https://datos.bne.es/resource/XX3280526,Relación histórica del viaje a la América Meri...,http://id.loc.gov/vocabulary/languages/spa,,,
freq,39,11,11,17,13,13,13


In [29]:
df.dtypes

author               object
work                 object
title                object
langCode             object
edition              object
yearOfPublication    object
placeOfProduction    object
dtype: object

In [30]:
sorted_df = df.sort_values(by='yearOfPublication')
print(sorted_df[["title","yearOfPublication"]].head(40))

                                                title yearOfPublication
38  Dissertación historica y greographica sobre el...                  
27  Observaciones astronómicas y físicas hechas en...                  
28  Dissertacion historica, y geographica sobre el...                  
29  Carta de Jorge Juan a Carlos III, en los últim...                  
30  Compendio de Navegación para el uso de los... ...                  
31  Lecciones de navegación para el uso de las com...                  
32  Noticias secretas de América sobre el estado n...                  
37                                    Correspondencia                  
26  Carta de Jorge Juan, desde Tetúan, al Gobernad...                  
33  Observaciones astronomicas, y phisicas hechas ...                  
34     Voyage historique de l'Amérique méridionale...                  
35  Voyage historique de l'Amerique meridionale fa...                  
36        Voyage historique de l'Amerique Meridionale           